## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

mnist_path = "./mnist.npz"

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data(path=mnist_path)
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[28*28, 100],
                                                         minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                         initial_value=tf.random.uniform(shape=[100, 10],
                                                         minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.7349437 ; accuracy 0.6217167
epoch 1 : loss 1.7312933 ; accuracy 0.62308335
epoch 2 : loss 1.7276508 ; accuracy 0.62438333
epoch 3 : loss 1.7240158 ; accuracy 0.62563336
epoch 4 : loss 1.720389 ; accuracy 0.62685
epoch 5 : loss 1.7167702 ; accuracy 0.62808335
epoch 6 : loss 1.7131591 ; accuracy 0.62895
epoch 7 : loss 1.7095561 ; accuracy 0.62993336
epoch 8 : loss 1.7059611 ; accuracy 0.63098335
epoch 9 : loss 1.7023741 ; accuracy 0.63208336
epoch 10 : loss 1.6987951 ; accuracy 0.63296664
epoch 11 : loss 1.6952242 ; accuracy 0.6339833
epoch 12 : loss 1.6916612 ; accuracy 0.63525
epoch 13 : loss 1.6881065 ; accuracy 0.63631666
epoch 14 : loss 1.68456 ; accuracy 0.6373
epoch 15 : loss 1.6810213 ; accuracy 0.6383833
epoch 16 : loss 1.6774912 ; accuracy 0.63951665
epoch 17 : loss 1.6739688 ; accuracy 0.64045
epoch 18 : loss 1.670455 ; accuracy 0.64163333
epoch 19 : loss 1.6669493 ; accuracy 0.643
epoch 20 : loss 1.6634517 ; accuracy 0.64411664
epoch 21 : loss 1.6599623 ; ac